In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For the dashboard
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Overview

In this notebook I have created a dashboard giving an overview of the problems in Zürich. The map shows the location of all problems, what the problem is and the status of the problem. The user can filter by the type of problem, to see only the problems of interest, e.g. only the problems with the type "Graffiti" that are still open.

### Why?/Usecases

Citizens can use this dashboard to see what problems there are in Zurich, to stay up to date.

If a citizen is planning a late picnic in Irchel Park, they can check the dashboard to see if there are any problems in the park, e.g.:

if the lighting is broken, they might have to bright a lamp themselves.
there was one (albeit erroneous) report of a type of mosquito that can carry the dangerous Dengue fever, so they might want to choose a different park.


Benevolent citizens can filter by cases of graffiti that are not fixed yet and clean it up themselves.

If you are moving to Wiedikon and don't know much about the area, you can check the dashboard to see what problems there are in the area, and if they are being fixed.


For the people behind "Zurich wie neu", this dashboard could maybe offer a different overview of the city's problems.


### Data

This data set is from "Zurich wie neu" (Zurich like new), a program where citizens can report problems or flaws in the city. Stadt Zurich will then respond to the report and ideally fix the problem.

The (important) columns in the data set are:

coordinates (e,n): the coordinates of the problem in Swiss (east, north) format

service_code: the type of problem in 10 categories

status: the status of the problem

requested_datetime: the date and time the problem was reported

updated_datetime: the date and time the problem was updated. If updated to "fixed" it indicates the time it took to fix the problem.

detail: a description of the problem from the citizen

media_url: a link to a picture of the problem uploaded by the citizen


### Analysis

The goal is to find some interesting insights in the data set and put it into a dashboard. The dashboard will be created using Plotly Dash.

### Dashboard Goals

The dashboard should:

Have a map of Zurich with the problems marked on it. The user should be able to select a problem and see the type, status, and when it was reported and updated.

Have a dropdown menu to filter by the type of problem.



In [30]:
data_raw['service_notice']

0        Diese Reparatur wird von uns in den kommenden ...
1        Diese Reparatur wird von uns in den kommenden ...
2        Diese Reparatur wird von uns in den kommenden ...
3        Diese Reparatur wird von uns in den kommenden ...
4        Dieses Graffiti wird von uns in den kommenden ...
                               ...                        
54140    Vielen Dank für Ihre Meldung. <br/>  <br/> Weg...
54141    Guten Abend <br/>  <br/> Das glauben wir Ihnen...
54142                                                  NaN
54143    Die Zweiradordnung wird Ihr Anliegen prüfen un...
54144    Vielen Dank für Ihre Meldung. <br/> Es gibt be...
Name: service_notice, Length: 54145, dtype: object

## Inspect data

In [26]:
data_raw = pd.read_csv('C:/Users/simon/Desktop/Code/Zuri_wie_neu/data/stzh_zwn_meldungen_p.csv')
data_raw.head()

,objectid,service_request_id,e,n,service_code,status,detail,media_url,interface_used,requested_datetime,agency_sent_datetime,updated_datetime,service_notice,title,description,service_name,url,geometry
0,1,1,2678968.0,1247548.0,Strasse/Trottoir/Platz,fixed - council,Auf dem Asphalt des Bürgersteigs hat es eine E...,NaN,Web interface,20130314151615,2.013040e+13,20130412075930,Diese Reparatur wird von uns in den kommenden ...,Auf dem Asp,Auf dem Asp: Auf dem Asphalt des Bürgersteigs ...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/1,POINT (2678968 1247548)
1,2,2,2680746.0,1249916.0,Strasse/Trottoir/Platz,fixed - council,Vermessungspunkt ist nicht mehr bündig mit dem...,NaN,Web interface,20130314151757,2.013033e+13,20130412080022,Diese Reparatur wird von uns in den kommenden ...,Vermessungs,Vermessungs: Vermessungspunkt ist nicht mehr b...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/2,POINT (2680746 1249916)
2,3,4,2684605.0,1251431.0,Strasse/Trottoir/Platz,fixed - council,Beim Trottoir sind einige Randsteine defekt un...,https://www.zueriwieneu.ch/photo/4.0.jpeg?bfbb...,Web interface,20130315091416,2.013032e+13,20130412080810,Diese Reparatur wird von uns in den kommenden ...,Beim Trotto,Beim Trotto: Beim Trottoir sind einige Randste...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/4,POINT (2684605 1251431)
3,4,5,2681754.0,1250376.0,Strasse/Trottoir/Platz,fixed - council,Auf dem Parkplatz beim Waidspital sind einige ...,https://www.zueriwieneu.ch/photo/5.0.jpeg?e309...,Web interface,20130315091715,2.013032e+13,20130412080905,Diese Reparatur wird von uns in den kommenden ...,Auf dem Par,Auf dem Par: Auf dem Parkplatz beim Waidspital...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/5,POINT (2681754 1250376)
4,5,6,2683094.0,1247762.0,Abfall/Sammelstelle,fixed - council,Arbeitskiste ist rund herum verschmiert,https://www.zueriwieneu.ch/photo/6.0.jpeg?8e65...,Web interface,20130315103653,2.013042e+13,20130423135033,Dieses Graffiti wird von uns in den kommenden ...,Arbeitskist,Arbeitskist: Arbeitskiste ist rund herum versc...,Abfall/Sammelstelle,https://www.zueriwieneu.ch/report/6,POINT (2683094 1247762)


## Data preprocessing

In [3]:
# Remove duplicate column
(data_raw['service_name'] == data_raw['service_code']).all()
data = data_raw.drop('service_code', axis=1)

In [4]:
# Convert timestamp to datetime format
date_columns = ['requested_datetime', 'agency_sent_datetime', 'updated_datetime']

for col in date_columns:
    data[col] = pd.to_datetime(data[col], format='%Y%m%d%H%M%S')

data[date_columns]

,requested_datetime,agency_sent_datetime,updated_datetime
0,2013-03-14 15:16:15,2013-04-04 07:25:05,2013-04-12 07:59:30
1,2013-03-14 15:17:57,2013-03-26 14:05:05,2013-04-12 08:00:22
2,2013-03-15 09:14:16,2013-03-15 09:55:05,2013-04-12 08:08:10
3,2013-03-15 09:17:15,2013-03-20 10:05:05,2013-04-12 08:09:05
4,2013-03-15 10:36:53,2013-04-22 18:25:05,2013-04-23 13:50:33
...,...,...,...
54140,2024-09-04 16:29:01,2024-09-04 16:32:05,2024-09-04 19:00:15
54141,2024-09-04 16:43:28,2024-09-04 16:47:04,2024-09-04 19:14:00
54142,2024-09-04 16:46:14,2024-09-04 17:22:04,2024-09-04 17:22:04
54143,2024-09-04 16:51:49,2024-09-04 16:52:05,2024-09-04 16:53:07


In [5]:
data['media_url'].fillna('No media', inplace=True)

In [6]:
# Add time it took to fix the problem
data[data['status'] == 'fixed - council'][['requested_datetime', 'updated_datetime']]
data['service_time'] = np.zeros(data.shape[0])
data['service_time'] = data['updated_datetime'] - data['requested_datetime']
data['service_time'] = data['service_time'].dt.total_seconds() / 60 / 60

# Add report_day as a column
data['report_day'] = data['requested_datetime'].dt.day_name()
# Add report_year
data['report_year'] = data['requested_datetime'].dt.year

# Ensure years and weekdays are ordered for plotting later
years = sorted(data['requested_datetime'].dt.year.unique())
weekdays = sorted(data['requested_datetime'].dt.day_name().unique(), key=lambda x: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(x))


Converting the coordinates from the swiss east and north format to latitude and longitude.

In [7]:
from pyproj import Transformer

# Define the transformer for converting CH1903+ / LV95 to WGS84
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)


# Function to transform coordinates
def transform_coordinates(row):
    longitude, latitude = transformer.transform(row['e'], row['n'])
    return pd.Series([longitude, latitude], index=['longitude', 'latitude'])

# Apply the function to each row in the DataFrame
data[['longitude', 'latitude']] = data.apply(transform_coordinates, axis=1)


In [8]:
# Clean up SZ answers to reports

In [9]:
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import Point
from pyproj import Transformer

# Define the transformer for converting CH1903+ / LV95 to WGS84
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

# Function to transform coordinates
def transform_coordinates(row):
    longitude, latitude = transformer.transform(row['e'], row['n'])
    return pd.Series([longitude, latitude], index=['longitude', 'latitude'])

# Apply the function to each row in the DataFrame
data[['longitude', 'latitude']] = data.apply(transform_coordinates, axis=1)


In [10]:
# Add media support: Show the picture if possible.
df = data[['longitude', 'latitude', 'status', 'requested_datetime', 'updated_datetime', 'service_time', 'service_name', 'report_day', 'report_year', 'detail', 'media_url', 'service_notice']]
df

,longitude,latitude,status,requested_datetime,updated_datetime,service_time,service_name,report_day,report_year,detail,media_url,service_notice
0,8.484226,47.374038,fixed - council,2013-03-14 15:16:15,2013-04-12 07:59:30,688.720833,Strasse/Trottoir/Platz,Thursday,2013,Auf dem Asphalt des Bürgersteigs hat es eine E...,No media,Diese Reparatur wird von uns in den kommenden ...
1,8.508193,47.395119,fixed - council,2013-03-14 15:17:57,2013-04-12 08:00:22,688.706944,Strasse/Trottoir/Platz,Thursday,2013,Vermessungspunkt ist nicht mehr bündig mit dem...,No media,Diese Reparatur wird von uns in den kommenden ...
2,8.559589,47.408259,fixed - council,2013-03-15 09:14:16,2013-04-12 08:08:10,670.898333,Strasse/Trottoir/Platz,Friday,2013,Beim Trottoir sind einige Randsteine defekt un...,https://www.zueriwieneu.ch/photo/4.0.jpeg?bfbb...,Diese Reparatur wird von uns in den kommenden ...
3,8.521627,47.399131,fixed - council,2013-03-15 09:17:15,2013-04-12 08:09:05,670.863889,Strasse/Trottoir/Platz,Friday,2013,Auf dem Parkplatz beim Waidspital sind einige ...,https://www.zueriwieneu.ch/photo/5.0.jpeg?e309...,Diese Reparatur wird von uns in den kommenden ...
4,8.538891,47.375455,fixed - council,2013-03-15 10:36:53,2013-04-23 13:50:33,939.227778,Abfall/Sammelstelle,Friday,2013,Arbeitskiste ist rund herum verschmiert,https://www.zueriwieneu.ch/photo/6.0.jpeg?8e65...,Dieses Graffiti wird von uns in den kommenden ...
...,...,...,...,...,...,...,...,...,...,...,...,...
54140,8.520284,47.403434,fixed - council,2024-09-04 16:29:01,2024-09-04 19:00:15,2.520556,Schädlinge,Wednesday,2024,Bei Waldspaziergang durch Tigermückenschwarm v...,https://www.zueriwieneu.ch/photo/60666.0.jpeg?...,Vielen Dank für Ihre Meldung. <br/> <br/> Weg...
54141,8.515094,47.379997,fixed - council,2024-09-04 16:43:28,2024-09-04 19:14:00,2.508889,Schädlinge,Wednesday,2024,Sehr geehrte Damen und Herren <br/> <br/> Sie...,https://www.zueriwieneu.ch/photo/60668.0.jpeg?...,Guten Abend <br/> <br/> Das glauben wir Ihnen...
54142,8.507043,47.352313,confirmed,2024-09-04 16:46:14,2024-09-04 17:22:04,0.597222,Grünflächen/Spielplätze,Wednesday,2024,"Flacher Wassertrog, ursprünglich zur Schuhrein...",No media,NaN
54143,8.524451,47.372560,fixed - council,2024-09-04 16:51:49,2024-09-04 16:53:07,0.021667,Abfall/Sammelstelle,Wednesday,2024,Diese KACK Velo versperrt seit Wochen den Velo...,https://www.zueriwieneu.ch/photo/60670.0.jpeg?...,Die Zweiradordnung wird Ihr Anliegen prüfen un...


# Adding Kreise to the data

In [11]:
from shapely import wkt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [12]:
kreis_df = pd.read_csv('C:/Users/simon/Desktop/Code/ZHkreise/data/stzh_adm_stadtkreise_a.csv')
kreis_df.rename(columns={"bezeichnung": "kreis_name"}, inplace=True)

kreis_df.sort_values('name', inplace=True)
kreis_df.drop(['objid','entstehung'], axis=1, inplace=True)
kreis_df.reset_index(drop=True, inplace=True)
kreis_df['geometry'] = kreis_df['geometry'].apply(wkt.loads)


transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

# Function to transform coordinates
def transform_coordinates(row):
    longitude, latitude = transformer.transform(row['e'], row['n'])
    return pd.Series([longitude, latitude], index=['longitude', 'latitude'])


def transform_polygon(polygon):
    # Transform all coordinates in the polygon
    coords = list(polygon.exterior.coords)
    transformed_coords = [transformer.transform(x, y) for x, y in coords]
    return Polygon(transformed_coords)

#Translate coordinates to lat,lon
kreis_df['geometry'] = kreis_df['geometry'].apply(transform_polygon)  



# Convert df's lat/lon to Shapely Point objects
df['point'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Function to classify each point based on polygons in kreis_df
def classify_point(point, polygons):
    for idx, polygon in enumerate(polygons):
        if polygon.contains(point):
            return idx
    return None


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\1609003335.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)


In [13]:
# Function to classify points based on polygons
def classify_point(row):
    for _, polygon_row in kreis_df.iterrows():
        if row['point'].within(polygon_row['geometry']):
            return polygon_row['name']
    return 'Unknown'

# Classify each point in the data
df['kreis_name'] = df.apply(classify_point, axis=1)

# Now 'data' DataFrame includes 'kreis_name' which indicates which kreis each point falls into.


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\407688397.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kreis_name'] = df.apply(classify_point, axis=1)


# Making the dashboard

In [14]:
# Making a style dictionary to easily and consistently change the style of the dashboard 
anthracite = '#393D47'
lightgray = '#d3d3d3'
white = '#ffffff'
black = '#000000'
blue = '#636efa'
font = 'Roboto, sans-serif'
mapbox_style = 'carto-positron', #['open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 'stamen-terrain']

style_dict = {
    'background_color': lightgray,
    'text_color': black,
    'accent_color': blue,
    'font': font,
    'mapbox_style': mapbox_style
    }

In [15]:
df

,longitude,latitude,status,requested_datetime,updated_datetime,service_time,service_name,report_day,report_year,detail,media_url,service_notice,point,kreis_name
0,8.484226,47.374038,fixed - council,2013-03-14 15:16:15,2013-04-12 07:59:30,688.720833,Strasse/Trottoir/Platz,Thursday,2013,Auf dem Asphalt des Bürgersteigs hat es eine E...,No media,Diese Reparatur wird von uns in den kommenden ...,POINT (8.484226301305455 47.374037733747315),9
1,8.508193,47.395119,fixed - council,2013-03-14 15:17:57,2013-04-12 08:00:22,688.706944,Strasse/Trottoir/Platz,Thursday,2013,Vermessungspunkt ist nicht mehr bündig mit dem...,No media,Diese Reparatur wird von uns in den kommenden ...,POINT (8.508192919637715 47.39511862093258),10
2,8.559589,47.408259,fixed - council,2013-03-15 09:14:16,2013-04-12 08:08:10,670.898333,Strasse/Trottoir/Platz,Friday,2013,Beim Trottoir sind einige Randsteine defekt un...,https://www.zueriwieneu.ch/photo/4.0.jpeg?bfbb...,Diese Reparatur wird von uns in den kommenden ...,POINT (8.559589406156757 47.40825905055263),12
3,8.521627,47.399131,fixed - council,2013-03-15 09:17:15,2013-04-12 08:09:05,670.863889,Strasse/Trottoir/Platz,Friday,2013,Auf dem Parkplatz beim Waidspital sind einige ...,https://www.zueriwieneu.ch/photo/5.0.jpeg?e309...,Diese Reparatur wird von uns in den kommenden ...,POINT (8.521627371871496 47.39913121917527),10
4,8.538891,47.375455,fixed - council,2013-03-15 10:36:53,2013-04-23 13:50:33,939.227778,Abfall/Sammelstelle,Friday,2013,Arbeitskiste ist rund herum verschmiert,https://www.zueriwieneu.ch/photo/6.0.jpeg?8e65...,Dieses Graffiti wird von uns in den kommenden ...,POINT (8.538890671691396 47.3754549474245),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54140,8.520284,47.403434,fixed - council,2024-09-04 16:29:01,2024-09-04 19:00:15,2.520556,Schädlinge,Wednesday,2024,Bei Waldspaziergang durch Tigermückenschwarm v...,https://www.zueriwieneu.ch/photo/60666.0.jpeg?...,Vielen Dank für Ihre Meldung. <br/> <br/> Weg...,POINT (8.520283960452161 47.40343443277984),10
54141,8.515094,47.379997,fixed - council,2024-09-04 16:43:28,2024-09-04 19:14:00,2.508889,Schädlinge,Wednesday,2024,Sehr geehrte Damen und Herren <br/> <br/> Sie...,https://www.zueriwieneu.ch/photo/60668.0.jpeg?...,Guten Abend <br/> <br/> Das glauben wir Ihnen...,POINT (8.515093730982437 47.379996803316224),4
54142,8.507043,47.352313,confirmed,2024-09-04 16:46:14,2024-09-04 17:22:04,0.597222,Grünflächen/Spielplätze,Wednesday,2024,"Flacher Wassertrog, ursprünglich zur Schuhrein...",No media,NaN,POINT (8.507043152473488 47.35231262125655),3
54143,8.524451,47.372560,fixed - council,2024-09-04 16:51:49,2024-09-04 16:53:07,0.021667,Abfall/Sammelstelle,Wednesday,2024,Diese KACK Velo versperrt seit Wochen den Velo...,https://www.zueriwieneu.ch/photo/60670.0.jpeg?...,Die Zweiradordnung wird Ihr Anliegen prüfen un...,POINT (8.524450819608292 47.372560228326954),4


To keep the code tidy, I define divs and graphs in separate and then call them in the app later.

In [23]:
kreis_list = np.sort([x if type(x) == int else 55 for x in df['kreis_name'].unique()])
kreis_list = [str(x) for x in kreis_list]
kreis_list = np.char.replace(kreis_list, '55', 'Unknown')

checklists_container = html.Div(
            id='checklists-container',
            style={'display':'flex', 'flexDirection':'row', 'alignItems': 'center', 'border-radius':'8px', 'width':'100%', 'padding': '2px'},
            children=[
                # Service name checklist
                html.H3("Select type", style={'margin-bottom': '2px', 'border-radius': '8px', 'color': style_dict['text_color'], 'backgroundColor': style_dict['background_color'], 'font-size': '16px'}),
                dcc.Checklist(
                    id='service-checklist',
                    options=[{'label': service, 'value': service} for service in df['service_name'].unique()],
                    value=df['service_name'].unique().tolist(),
                    labelStyle={'display': 'block', 'margin-bottom': '2px', 'font-size': '16px'}
                ),
                
                # Status checklist
                html.H3("Select Status", style={'margin-top': '2px', 'margin-bottom': '2px', 'border-radius': '8px','color': style_dict['text_color'], 'backgroundColor': style_dict['background_color'], 'font-size': '16px'}),
                dcc.Checklist(
                    id='status-checklist',
                    options=[{'label': status, 'value': status} for status in df['status'].unique()],
                    value=df['status'].unique().tolist(),
                    labelStyle={'display': 'block', 'margin-bottom': '2px', 'font-size': '16px'}
                ),

                # Year and day filter
                html.H3("Select Year and Weekday", style={'margin-top': '4px', 'margin-bottom': '2px', 'border-radius': '8px', 'color': style_dict['text_color'], 'backgroundColor': style_dict['background_color'], 'font-size': '16px'}),
                dcc.Checklist(
                    id='year-weekday-checklist',
                    options=[
                        {'label': str(year), 'value': str(year)} for year in years
                    ] +
                    [{'label': day, 'value': day} for day in weekdays],
                    value=[str(year) for year in years] +
                          [day for day in weekdays],
                    labelStyle={'display': 'block', 'margin-bottom': '2px', 'font-size': '16px'}
                ),

                html.H3("Select Kreis", style={'margin-top': '2px', 'margin-bottom': '2px', 'border-radius': '8px','color': style_dict['text_color'], 'backgroundColor': style_dict['background_color'], 'font-size': '16px'}),
                dcc.Checklist(
                    id='kreis-checklist',
                    options=[{'label': kreis, 'value': kreis} for kreis in kreis_list],
                    value=kreis_list,
                    labelStyle={'display': 'block', 'margin-bottom': '2px', 'font-size': '16px'}
                ),
            ]
        )



checklist_button = html.Button('Show/Hide Filters', id='toggle-button', n_clicks=0, style={'border-radius': '8px', 'padding': '10px', 'background-color': style_dict['accent_color'], 'color': style_dict['text_color']})



fixed_container = html.Div(
            id='summary-box',
            style={
                'padding': '4px',
                'border': '4px solid #ccc',
                'margin': '4px',
                'color': style_dict['text_color'],
                'backgroundColor': style_dict['background_color'],
                'textAlign': 'center',
                'font-size': '21px',
                'border-radius': '8px',
            }
        )



map_and_bars_container = html.Div(
            style={'display': 'flex', 'justifyContent': 'space-between'},
            children=[
                dcc.Graph(
                    id='map',
                    style={'border-radius': '8px', 'flex':1, 'fontFamily': style_dict['font'], 'backgroundColor': style_dict['background_color']}
                ),
                html.Div(
                    style={'width': '35%', 'display': 'flex', 'border-radius': '8px', 'flexDirection': 'column', 'justifyContent': 'space-between'},
                    children=[
                        html.Div(
                            id='detail-box',
                            style={'border-radius': '8px', 'flex': 1, 'padding': '10px', 'fontFamily': style_dict['font'], 'backgroundColor': style_dict['background_color']}
                        ),
                        dcc.Graph(
                            id='service-bar',
                            style={'border-radius': '8px', 'flex':1, 'fontFamily': style_dict['font'], 'backgroundColor': style_dict['background_color']}
                        )
                    ]
                )
            ]
        )



time_series_container = html.Div(
            style={'display': 'flex', 'justifyContent': 'space-between', 'border-radius': '8px'},
            children=[
                
                dcc.Graph(
                    id='time-series',
                    style={'width': '100%', 'height': '40vh', 'border': '4px solid #ccc', 'padding': '2px', 'fontFamily': style_dict['font'], 'backgroundColor': style_dict['background_color']}
        )]

        )



header_container = html.H1("Zurich Wie Neu - All reports", style={'textAlign': 'center', 'border-radius': '8px','color': style_dict['text_color'], 'margin': '1px', 'font-size': '24px'})

#detail_container = html.Div(id='detail-box', children=[], style={'padding': '10px', 'border': '1px solid #ddd', 'border-radius': '5px'})

list_of_containers = [header_container, checklist_button, checklists_container, fixed_container, map_and_bars_container, time_series_container, html.Link(
            href='https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap', rel='stylesheet'),]

# Make the dashboard

In [24]:
# Initialize the Dash app
import plotly.graph_objects as go
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div(
    style={'backgroundColor': style_dict['background_color'], 'border-radius': '8px','padding': '4px', 'fontFamily': style_dict['font']},
    children=list_of_containers
)

def toggle_checklists(n_clicks):
    if n_clicks % 2 == 0:
        return {'display': 'none'}
    else:
        return {'display': 'block'}
    

@app.callback(
    [Output('checklists-container', 'style'), 
     Output('map', 'figure'), 
     Output('time-series', 'figure'), 
     Output('summary-box', 'children'),
     Output('detail-box', 'children'),
     Output('service-bar', 'figure'),
],

    [Input('toggle-button', 'n_clicks'),
     Input('service-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('year-weekday-checklist', 'value'),
     Input('kreis-checklist', 'value'),
     Input('map', 'clickData')
    ],
)


def update_dashboard(n_clicks, selected_services, selected_statuses, selected_years_days, selected_kreise, clickData):
    #Filter the data based on the checklist values
    print('Go')
    print(clickData)
    print('check')
    
    ########################################################################################
    #################################### FILTERING #########################################
    ########################################################################################


    # Extract selected years and days
    selected_years = [x for x in selected_years_days if x.isdigit()]  # Years are digits
    selected_days  = [x for x in selected_years_days if not x.isdigit()] # Days are not digits

    # Convert years to integers since requested_datetime.year is numeric
    selected_years = list(map(int, selected_years))
    
    # Filter the DataFrame based on selected service types, statuses, years, and days
    filtered_df = df[df['service_name'].isin(selected_services) &
                     df['status'].isin(selected_statuses) &
                     df['requested_datetime'].dt.year.isin(selected_years) &
                     df['requested_datetime'].dt.day_name().isin(selected_days) &
                     df['kreis_name'].isin(selected_kreise)]

    default_figure = {
        'data': [],
        'layout': {
            'title': 'No Data Available',
            'xaxis': {'title': 'X-axis'},
            'yaxis': {'title': 'Y-axis'}
            }
        }

    default_text = "No data available with the selected filters."

    if filtered_df.empty:
        return (
            {'display': 'block' if n_clicks % 2 else 'none'},  # Checklists container visibility
            default_figure,  # Map figure
            default_figure,  # Time series figure
            default_text,    # Summary box content
            default_figure,  # Weekday bar figure
            default_figure,  # Service bar figure
            default_text  # Default detail message
        )

    ########################################################################################
    ################################# ADDING KREISE ########################################
    ########################################################################################

    polygons = []
    colors = px.colors.qualitative.Plotly
    color_map = {}
    color_index = 0

    for _, row in kreis_df.iterrows():
        polygon = row['geometry']
        # Check if polygon.exterior.coords is a generator
        coords = list(polygon.exterior.coords)

        # Unpack coordinates
        lon, lat = zip(*coords)

        # Ensure lat and lon are lists and close the polygon
        lat = list(lat) + [lat[0]]
        lon = list(lon) + [lon[0]]

        name = row['name']
        if name not in color_map:
            color_map[name] = colors[color_index % len(colors)]
            color_index += 1

        # Append the polygon as a trace to the list
        polygons.append({
            'lat': lat,
            'lon': lon,
            'name': name,
            'color': color_map[name]
        })

    # Convert polygons list to DataFrame
    polygon_df = pd.DataFrame(polygons)


    # Create a scatter mapbox plot for the map
    fig_map = px.scatter_mapbox(
        filtered_df,
        lat='latitude',
        lon='longitude',
        color='service_name',
        hover_name='service_name',
        hover_data={
            'longitude':False,
            'latitude':False,
            'service_name': True,
            'status': True,
            'requested_datetime': True,
            'detail': False,
            'kreis_name': True,
            },
#        data_frame={'longitude', 'latitude'},
        zoom=11,
        mapbox_style='carto-positron'
    )
    

######################################################################################
    
    fig_map.update_layout(
        title='Map of Reports in Zürich',
#        mapbox_style=style_dict['mapbox_style'],
        clickmode='event+select', 
        font=dict(family=style_dict['font']),
        hovermode='closest',
        hoverdistance=2,
        uirevision = 'foo', # So that the map does not reset when unselecting a point
    )

    fig_map.update_traces(
        marker=dict(size=10),
        hovertemplate="<br>".join([
            #"Detail:</b> %{customdata[0]}",
            "<b>Service:</b> %{customdata[1]}",
            "<b>Status:</b> %{customdata[2]}",
            "<b>Request time:</b> %{customdata[3]}",
            "<b>Media url:</b> %{customdata[4]}",
            "<b>Kreis:</b> %{customdata[5]}",
        ]),
        customdata = filtered_df[['detail','service_name', 'status', 'requested_datetime', 'media_url', 'kreis_name','service_notice']]
    )

        # Add polygons to the map
    for _, poly in polygon_df.iterrows():
        fig_map.add_scattermapbox(
            lat=poly['lat'],
            lon=poly['lon'],
            hoverinfo='skip',
            mode='lines',
            line=dict(width=2, color=poly['color']),  # Adjust the line style as needed
            name='kreis '+str(poly['name'],
            )
        )


    ########################################################################################
    ################################ TIMES SERIES PLOT #####################################
    ########################################################################################


    # Create a time series plot for the number of reports over time
    filtered_df['date'] = filtered_df['requested_datetime'].dt.date
    report_counts = filtered_df['date'].value_counts().sort_index()
    fig_time_series = px.line(report_counts,
        x=report_counts.index,
        y=report_counts.values,
        labels={'x': 'Date', 'y': 'Number of Reports'},
        title='Number of Reports Over Time'
    )
    fig_time_series.update_layout(font=dict(family=style_dict['font']))

    # Create bar graphs for weekdays and service names
    weekday_counts = filtered_df['requested_datetime'].dt.day_name().value_counts().reindex(weekdays)
    fig_weekday_bar = px.bar(
        weekday_counts,
        x=weekday_counts.index,
        y=weekday_counts.values,
        labels={'x': 'Weekday', 'y': 'Number of Reports'},
        title='Reports by Weekday'
    )
    fig_weekday_bar.update_layout(font=dict(family=style_dict['font']))

    service_counts = filtered_df['service_name'].value_counts()
    fig_service_bar = px.bar(
        service_counts,
        x=service_counts.index,
        y=service_counts.values,
        labels={'x': 'Service Name', 'y': 'Number of Reports'},
        title='Reports by Service Name'
    )
    fig_service_bar.update_layout(font=dict(family=style_dict['font']))

    # Calculate average service time
    avg_service_time = filtered_df['service_time'].mean()
    # Calculate percentage of "fixed - council" status
    percent_fixed_council = (filtered_df['status'] == 'fixed - council').mean() * 100

    # Create the summary box content
    summary_text = f"Average Service Time: {avg_service_time:.2f} hours | Percentage 'fixed - council': {percent_fixed_council:.2f}%"

    ########################################################################################
    ############################### MAKE MAP CLICKABLE #####################################
    ########################################################################################


    if clickData is None:
        detail_text = "Click on a point to see the report details."
        image_url   = "Click on a point to see the report image."
        SZ_text     = "Click on a point to see the report response."
        detail_content = html.Div([
            html.P(detail_text),
            html.Img(src=image_url, style={'width': '100%', 'border-radius': '8px'}),  # Display image
            html.P(SZ_text)
            ])
    else:
        # Extract 'detail' and 'image_url' from clickData
        detail_text = clickData['points'][0]['customdata'][0]  # 'detail' is in customdata
        image_url   = clickData['points'][0]['customdata'][4]  # Assuming 'image_url' is in customdata as well
        SZ_text     = clickData['points'][0]['customdata'][5]  # Assuming 'SZ_text' is in customdata as well

        # Build content for the detail box
        detail_content = html.Div([
            html.P(detail_text),
            html.Img(src=image_url, style={'width': '100%', 'border-radius': '8px'}.
            html.P(SZ_text))  # Display image
        ])

    
    return (
        {'display': 'block' if n_clicks % 2 else 'none'},  # Checklists container visibility
        fig_map,  # Map figure
        fig_time_series,  # Time series figure
        summary_text,     # Summary box content
        detail_content,   # Detail box content
        fig_service_bar,  # Service bar figure
    )



port = '8051'
print('url: http://127.0.0.1:'+port+'/')
if __name__ == '__main__':
    app.run_server(debug=True, port=port)


url: http://127.0.0.1:8051/


Go
None
check


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\2844408599.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Go
None
check


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\2844408599.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

Go
None
check


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\2844408599.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Go
None
check


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\2844408599.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Go
None
check
---------------------------------------------------------------------------
SchemaLengthValidationError               Traceback (most recent call last)
SchemaLengthValidationError: Schema: [<Output `checklists-container.style`>, <Output `map.figure`>, <Output `time-series.figure`>, <Output `summary-box.children`>, <Output `detail-box.children`>, <Output `service-bar.figure`>]
Path: ()
Expected length: 6
Received value of length 7:
    [{'display': 'block'}, {'data': [], 'layout': {'title': 'No Data Available', 'xaxis': {'title': 'X-axis'}, 'yaxis': {'title': 'Y-axis'}}}, {'data': [], 'layout': {'title': 'No Data Available', 'xaxis': {'title': 'X-axis'}, 'yaxis': {'title': 'Y-axis'}}}, 'No data available with the selected filters.', {'data': [], 'layout': {'title': 'No Data Available', 'xaxis': {'title': 'X-axis'}, 'yaxis': {'title': 'Y-axis'}}}, {'data': [], 'layout': {'title': 'No Data Available', 'xaxis': {'title': 'X-axis'}, 'yaxis': {'title': 'Y-axis'}}}, 'No data ava

## To do

Fix color coding

Fix service_name appearing twice (one random)

kreis_name appears in kreis_df and df

## Ideas 

Split into Kreisen

Look into response time

Forecast reports 2013-2022 and compare to actual reports 2022-2024. Perhaps forecast all categories individually and then combine

Look at graffiti (only crime column)

Make dashboard with map

Percentage fixed as a function of service code

Which service codes take the longest to fix?

There is a seasonality in requests. Predict requests for 2022-2023 and compare to actual requests.

Find most graffiti prone areas. Forecast maybe?

Make word clouds for each service type

10. Interactive Timeline of Major Events

    Create an interactive timeline that plots major events (e.g., spikes in certain problem types, resolution of high-profile issues, etc.).
    This can provide context around why certain trends or patterns may have occurred (e.g., weather events, public holidays, etc.).


Group reports by week or months in timeseries plot 

# Predict reports

In [19]:
df1['date']

NameError: name 'df1' is not defined

Go
None
check
Go
None
check


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\1743375409.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\simon\AppData\Local\Temp\ipykernel_16368\1743375409.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Go
None
check


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\1743375409.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

Go
{'points': [{'curveNumber': 0, 'pointNumber': 0, 'pointIndex': 0, 'lon': 8.582868073483983, 'lat': 47.350794497859106, 'hovertext': 'Graffiti', 'bbox': {'x0': 534.4064462479413, 'x1': 536.4064462479413, 'y0': 545.9959740889283, 'y1': 547.9959740889283}, 'customdata': ['Der Stromkasten bei der S18 Station Zürich Rehalp wurde mit tags vermalt und besprüht.', 'Graffiti', 'fixed - council', '2019-10-13T18:07:22', 'https://www.zueriwieneu.ch/photo/21901.0.jpeg?ed0c1687', 'Unknown', 'Besten Dank für Ihre Meldung auf «Züri wie neu». Die Schmierereien werden so bald als möglich entfernt. <br/>  <br/> Freundliche Grüsse <br/> Ihre Stadt Zürich <br/>         ']}]}
check


C:\Users\simon\AppData\Local\Temp\ipykernel_16368\1743375409.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Input In [17], in update_dashboard(
    n_clicks=2,
    selected_services=['Strasse/Trottoir/Platz', 'Abfall/Sammelstelle', 'Grünflächen/Spielplätze', 'Beleuchtung/Uhren', 'Graffiti', 'Signalisation/Lichtsignal', 'Brunnen/Hydranten', 'Allgemein', 'VBZ/ÖV', 'Schädlinge'],
    selected_statuses=['fixed - council', 'external', 'jurisdiction unk', 'not contactable', 'wish', 'confirmed'],
    selected_years_days=['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    selected_kreise=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 'Unknown', 'Kreis 6'],
    clickData={'points': [{'bbox': {'x0': 534.4064462479413, 'x1': 536.4064462479413, 'y0': 545.9959740889283, 'y1': 547.9959740889283}, 'curveNumber': 0,

C:\Users\simon\AppData\Local\Temp\ipykernel_16368\1743375409.py:188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Input In [17], in update_dashboard(
    n_clicks=3,
    selected_services=['Strasse/Trottoir/Platz', 'Abfall/Sammelstelle', 'Grünflächen/Spielplätze', 'Beleuchtung/Uhren', 'Graffiti', 'Signalisation/Lichtsignal', 'Brunnen/Hydranten', 'Allgemein', 'VBZ/ÖV', 'Schädlinge'],
    selected_statuses=['fixed - council', 'external', 'jurisdiction unk', 'not contactable', 'wish', 'confirmed'],
    selected_years_days=['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    selected_kreise=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 'Unknown', 'Kreis 6'],
    clickData={'points': [{'bbox': {'x0': 534.4064462479413, 'x1': 536.4064462479413, 'y0': 545.9959740889283, 'y1': 547.9959740889283}, 'curveNumber': 0,

### Split data into train, val, test based on time.

In [70]:
df1 = df[['requested_datetime', 'service_name']]

# Extract time-based features
df1['day_of_week'] = df1['datetime'].dt.dayofweek
df1['day']   = df1['datetime'].dt.day
df1['month'] = df1['datetime'].dt.month
df1['year']  = df1['datetime'].dt.year

# Cyclical encoding for 'hour' and 'day_of_week'
df1['day_of_week_sin'] = np.sin(2 * np.pi * df1['day_of_week'] / 7)
df1['day_of_week_cos'] = np.cos(2 * np.pi * df1['day_of_week'] / 7)

# Time difference between consecutive reports
df1['time_diff'] = df1['datetime'].diff().dt.total_seconds().fillna(0)
df1['service_name'] = df1['service_name'].astype('category').cat.codes


train_split_end = '2022-01-01'
val_split_end = '2023-01-01'

train_data = df1[df['requested_datetime'] < train_split_end]#.values
val_data   = df1[(df['requested_datetime'] > train_split_end) & (df['requested_datetime'] < val_split_end)].values
test_data  = df1[(df['requested_datetime'] > val_split_end)].values
print(train_data.shape, val_data.shape, test_data.shape)


KeyError: 'datetime'

In [69]:
train_data

,requested_datetime,service_name
0,2013-03-14 15:16:15,8
1,2013-03-14 15:17:57,8
2,2013-03-15 09:14:16,8
3,2013-03-15 09:17:15,8
4,2013-03-15 10:36:53,0
...,...,...
31755,2021-12-31 14:32:39,0
31756,2021-12-31 14:33:41,0
31757,2021-12-31 14:34:31,0
31758,2021-12-31 17:29:02,0


In [ ]:

scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)
val_data_scaled = scaler.transform(val_data)
test_data_scaled = scaler.transform(test_data)

# Define a function to create the LSTM dataset
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), :])
        y.append(data[i + time_steps, :])
    return np.array(X), np.array(y)


# Set time steps (number of previous time steps to use for prediction)
time_steps = 12  # Example: Using the previous 12 time steps to predict the next one

# Create the LSTM dataset for training
X_train, y_train = create_dataset(train_scaled, time_steps)

# Reshape input data to be 3D (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])

X_test, y_test = create_dataset(test_scaled, time_steps)

# Reshape input data to be 3D (samples, time steps, features)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

In [ ]:
# Split data into before and after 2015
split_date = '2022-01-01'
train_data = df.loc['1950-01-01':split_date]
test_data = df.loc[split_date:'2024-09-05']

# Prepare the data
train_values = train_data[['PRCP', 'TMAX', 'TMIN']].values
test_values  = test_data[['PRCP', 'TMAX', 'TMIN']].values

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_values)
test_scaled = scaler.transform(test_values)

# Define a function to create the LSTM dataset
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), :])
        y.append(data[i + time_steps, :])
    return np.array(X), np.array(y)

# Set time steps (number of previous time steps to use for prediction)
time_steps = 12  # Example: Using the previous 12 time steps to predict the next one

# Create the LSTM dataset for training
X_train, y_train = create_dataset(train_scaled, time_steps)

# Reshape input data to be 3D (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])

X_test, y_test = create_dataset(test_scaled, time_steps)

# Reshape input data to be 3D (samples, time steps, features)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import plotly.graph_objects as go

# Assume 'df' is your DataFrame with columns 'requested_datetime' and 'service_name'
# Convert requested_datetime to date
df1 = df.copy()
df1['date'] = pd.to_datetime(df1['requested_datetime'].dt.date)

# Group by date and service_name to get the number of reports per service type per day
df_grouped = df1.groupby(['date', 'service_name']).size().reset_index(name='num_reports')

# Pivot the data to create columns for each service_name
df_pivot  = df_grouped.pivot(index='date', columns='service_name', values='num_reports').fillna(0).reset_index()

# Add the total number of reports per day as a separate column
df_pivot['total_reports'] = df_pivot.drop(columns='date').sum(axis=1)

# Split the data into training (until end of 2021), validation (2022), and testing (2023-2024)
train_data = df_pivot[df_pivot['date'] < '2022-01-01']
val_data   = df_pivot[(df_pivot['date'] >= '2022-01-01') & (df_pivot['date'] < '2023-01-01')]
test_data  = df_pivot[(df_pivot['date'] >= '2023-01-01') & (df_pivot['date'] <= '2024-01-01')]

# Prepare the data for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale each set independently to prevent data leakage
train_scaled = scaler.fit_transform(train_data.drop(columns='date'))
val_scaled = scaler.fit_transform(val_data.drop(columns='date'))
test_scaled = scaler.transform(test_data.drop(columns='date'))

# Function to create sequences for LSTM
def create_sequences(data, sequence_length=30):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), :])   # Use all columns except the last one for prediction
        y.append(data[i + sequence_length, -1])      # Predict the 'total_reports' column
    return np.array(X), np.array(y)

# Create sequences
sequence_length = 32
X_train, y_train = create_sequences(train_scaled, sequence_length)
X_val, y_val = create_sequences(val_scaled, sequence_length)
X_test, y_test = create_sequences(test_scaled, sequence_length)

# Reshape input for LSTM (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], X_val.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Early stopping callback with validation loss
early_stopping = EarlyStopping(
    monitor='val_loss',  # Use validation loss for early stopping
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

# Build LSTM model using Input layer
model = Sequential()
model.add(Input(shape=(sequence_length, X_train.shape[2])))  # Define input shape here
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))  # Add dropout to prevent overfitting
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))  # Add dropout to prevent overfitting
model.add(Dense(25))
model.add(Dense(1))

# Compile the model with a smaller learning rate
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with validation data
history = model.fit(
    X_train, y_train, 
    batch_size=32, 
    epochs=100, 
    validation_data=(X_val, y_val),  # Specify validation data
    callbacks=[early_stopping]
)

# Predict using the test set
test_predictions = model.predict(X_test)

# Plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_data['date'][sequence_length:], y=y_test, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_data['date'][sequence_length:], y=test_predictions.flatten(), mode='lines', name='Predicted'))
fig.update_layout(title='Predicted vs Actual Reports (2023-2024)', xaxis_title='Date', yaxis_title='Number of Reports')
fig.show()


Epoch 1/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 0.0041 - val_loss: 0.0045
Epoch 2/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0025 - val_loss: 0.0044
Epoch 3/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0022 - val_loss: 0.0045
Epoch 4/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0022 - val_loss: 0.0048
Epoch 5/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0024 - val_loss: 0.0045
Epoch 6/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0022 - val_loss: 0.0049
Epoch 7/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0023 - val_loss: 0.0045
Epoch 8/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.0023 - val_loss: 0.0045
Epoch 9/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0024 - val_loss: 0.0046
Epoch 10/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0026 - val_loss: 0.0050
Epoch 11/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0022 - val_loss: 0.0049
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step


In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import plotly.graph_objects as go


# Assume 'df' is your DataFrame with columns 'requested_datetime' and 'service_name'
# Group by date to get the number of reports per day
df1 = df.copy()
df1['date'] = pd.to_datetime(df1['requested_datetime'].dt.date)  # Convert to date only
df_grouped = df1.groupby('date').size().reset_index(name='num_reports')

# Split the data into training (until 2022) and testing (2022-2024)
train_data = df_grouped[df_grouped['date'] < '2022-01-01']
test_data = df_grouped[(df_grouped['date'] >= '2022-01-01') & (df_grouped['date'] <= '2024-01-01')]

# Prepare the data for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale training data
train_scaled = scaler.fit_transform(train_data[['num_reports']])
test_scaled = scaler.transform(test_data[['num_reports']])

# Function to create sequences for LSTM
def create_sequences(data, sequence_length=30):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), 0])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)

# Create sequences
sequence_length = 32
X_train, y_train = create_sequences(train_scaled, sequence_length)
X_test, y_test   = create_sequences(test_scaled, sequence_length)

# Reshape input for LSTM (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test  = np.reshape(X_test,  (X_test.shape[0], X_test.shape[1], 1))



early_stopping = EarlyStopping(
    monitor='loss',             # or 'val_accuracy', depending on your metric
    patience=5,                 # Number of epochs with no improvement after which training will be stopped
    min_delta=0.001,            # Minimum change to qualify as an improvement
    restore_best_weights=True   # Restore model weights from the epoch with the best value of the monitored quantity
)


# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


# Train the model
if True:
    model.fit(X_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping])


Go
None
check
Go
None
check


C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [17]:

# Predict using the model
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse transform predictions
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)
# Ensure 'predictions' column is initialized
train_data['predictions'] = np.nan
test_data['predictions'] = np.nan

# Assign train predictions (ensure alignment)
train_data.loc[train_data.index[sequence_length:], 'predictions'] = train_predictions.flatten()

# Assign test predictions (ensure alignment)
test_data.loc[test_data.index[sequence_length:], 'predictions'] = test_predictions.flatten()

# Combine the dataframes for visualization
combined_df = pd.concat([train_data, test_data])

# Visualization
fig = go.Figure()

# Add actual historical data
fig.add_trace(go.Scatter(
    x=train_data['date'],
    y=train_data['num_reports'],
    mode='lines',
    name='Historical Data (Train)',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=test_data['date'],
    y=test_data['num_reports'],
    mode='lines',
    name='Historical Data (Test)',
    line=dict(color='green')
))

# Add forecasted data
fig.add_trace(go.Scatter(
    x=combined_df['date'],
    y=combined_df['predictions'],
    mode='lines',
    name='Forecasted Data',
    line=dict(color='red', dash='dash')
))

# Update layout
fig.update_layout(
    title='Forecast of Number of Reports Over Time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Number of Reports'),
    font=dict(family='Roboto'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()


99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


C:\Users\simon\AppData\Local\Temp\ipykernel_196\2377552096.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\simon\AppData\Local\Temp\ipykernel_196\2377552096.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
# Filter data from 2022-2024 for actual and predicted values
test_data_2022_2024 = test_data[test_data['date'].between('2022-01-01', '2024-12-31')]

# Initialize a figure
fig = go.Figure()

# Plot the actual data from 2022-2024
fig.add_trace(go.Scatter(
    x=test_data_2022_2024['date'],
    y=test_data_2022_2024['num_reports'],
    mode='lines+markers',
    name='Actual Data (2022-2024)',
    line=dict(color='green')
))

# Plot the predicted data from 2022-2024
fig.add_trace(go.Scatter(
    x=test_data_2022_2024['date'],
    y=test_data_2022_2024['predictions'],
    mode='lines+markers',
    name='Predicted Data (2022-2024)',
    line=dict(color='red', dash='dash')  # Dashed line for predictions
))

# Update the layout
fig.update_layout(
    title='Actual vs Predicted Reports (2022-2024)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Number of Reports'),
    font=dict(family='Roboto'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()


In [19]:
df['requested_datetime']

0       2013-03-14 15:16:15
1       2013-03-14 15:17:57
2       2013-03-15 09:14:16
3       2013-03-15 09:17:15
4       2013-03-15 10:36:53
                ...        
54140   2024-09-04 16:29:01
54141   2024-09-04 16:43:28
54142   2024-09-04 16:46:14
54143   2024-09-04 16:51:49
54144   2024-09-04 17:14:59
Name: requested_datetime, Length: 54145, dtype: datetime64[ns]